In [2]:
from smartcard import *
from smartcard.CardType import *
from smartcard.CardRequest import *
from smartcard.CardConnectionObserver import *
from smartcard.util import *

In [8]:
cardtype = AnyCardType()
cardrequest = CardRequest( timeout=10, cardType=cardtype )
cardservice = cardrequest.waitforcard()

In [9]:
cardservice.connection.connect()
print (toHexString( cardservice.connection.getATR() ))

3B 00


In [10]:
print(cardservice.connection.getReader())

ACS ACR122U PICC Interface 00 00


In [11]:
SELECT = [0xA0, 0xA4, 0x00, 0x00, 0x02]
DF_TELECOM = [0x7F, 0x10]

In [12]:
data, sw1, sw2 = cardservice.connection.transmit( SELECT + DF_TELECOM )

IndexError: list index out of range

In [13]:
a = [1,5,8]
a[-2]

5